# Place a molecule onto a Surface

In [ ]:
%aiida

In [ ]:
# General imports
import nglview
import numpy as np
import ipywidgets as ipw
from collections import OrderedDict
from ase.data import vdw_radii
from IPython.display import display, clear_output, HTML
from numpy.linalg import norm
import scipy.stats

# AiiDA imports
from aiida.engine import calcfunction
from aiida.orm import StructureData
from aiidalab_widgets_base import StructureBrowserWidget
from apps.surfaces.widgets import analyze_structure

from ase.data import covalent_radii
from ase.neighborlist import NeighborList
import ase.neighborlist
import ase.build


# Custom imports
from apps.surfaces.widgets import slabs

## Step 1: Select a structure from the AiiDA database

In [ ]:
def on_struct_change(c):
    global orig_structure
    orig_structure = None # disable event processing
    s = struct_browser.results.value
    if s:
        atoms = s.get_ase()
        details=analyze_structure.analyze(atoms)
        orig_structure = s
    update_view()

In [ ]:
struct_browser = StructureBrowserWidget()
struct_browser.results.observe(on_struct_change, names='value')
display(struct_browser)
style = {'description_width': '30px'}
layout = {'width': '10%'}

## Step 2: Select cell size

In [ ]:
def on_nxyz_change(c):
    update_view()
    


selection = ipw.Text(placeholder='1..10',
                                    description='selection',
                                    style=style, layout={'width': '60%'})
normal =  ipw.Text(placeholder='0 0 1',
                                    description='axis rot.',
                                    style=style, layout={'width': '30%'})
phi_slider = ipw.BoundedFloatText(description='Dphi:', value=30, min=-90, 
                                        max=90, step=5,style=style, layout=layout)
phi_slider.observe(on_nxyz_change, names='value')

dx_slider = ipw.BoundedFloatText(description='Dx:', value=0.1, min=0.01, 
                                        max=2, step=0.1,style=style, layout=layout)
dx_slider.observe(on_nxyz_change, names='value')

dy_slider = ipw.BoundedFloatText(description='Dy:', value=0.1, min=0.01, 
                                        max=2, step=0.1,style=style, layout=layout)
dy_slider.observe(on_nxyz_change, names='value')

dz_slider = ipw.BoundedFloatText(description='Dz:', value=0.1, min=0.01, 
                                        max=2, step=0.1,style=style, layout=layout)
dz_slider.observe(on_nxyz_change, names='value')

dr_slider = ipw.BoundedFloatText(description='Dr:', value=0.1, min=0.01, 
                                        max=2, step=0.1,style=style, layout=layout)
dr_slider.observe(on_nxyz_change, names='value')

#display(drop_surface,dx_slider,dy_slider,dz_slider,nx_slider, ny_slider, nz_slider,phi_slider)


display(ipw.VBox([selection,
                  ipw.HBox([dx_slider,dy_slider,dz_slider]),
                   ipw.HBox([dr_slider,phi_slider,normal])]))

In [ ]:
def update_view():

    with info_out:
        clear_output()

        inp_descr.value = ""

        # remove old components
        if hasattr(viewer, "component_1"):
            viewer.component_1.clear_representations()
            viewer.component_1.remove_unitcell()
            cid = viewer.component_1.id
            viewer.remove_component(cid)
        if hasattr(viewer, "component_0"):
            viewer.component_0.clear_representations()
            viewer.component_0.remove_unitcell()
            cid = viewer.component_0.id
            viewer.remove_component(cid)

        if orig_structure:
            phi = phi_slider.value
            dx = dx_slider.value
            dy = dy_slider.value
            dz = dz_slider.value
            inp_descr.value = 'TEST'
            mol = orig_structure.get_ase()
            slab = slabs.prepare_slab(mol,
                                dx=dx,dy=dy,dz=dz,phi=phi, nx=nx, ny=ny, nz=nz,which_surf=which_surf)
            atoms = mol + slab
#From Kristjan    
            viewer.add_component(nglview.ASEStructure(mol)) # adds ball+stick
            viewer.add_component(nglview.ASEStructure(slab), default_representation=False)
            viewer.add_unitcell()
            viewer.center()

            viewer.component_1.add_ball_and_stick(aspectRatio=10.0, opacity=1.0)


            # Orient camera to look from positive z
            cell_z = atoms.cell[2, 2]
            com = atoms.get_center_of_mass()
            def_orientation = viewer._camera_orientation
            top_z_orientation = [1.0, 0.0, 0.0, 0,
                                 0.0, 1.0, 0.0, 0,
                                 0.0, 0.0, -np.max([cell_z, 30.0]) , 0,
                                 -com[0], -com[1], -com[2], 1]
            viewer._set_camera_orientation(top_z_orientation)

In [ ]:
viewer = nglview.NGLWidget()
info_out = ipw.Output()
display(viewer, info_out)

## Step 3: Store structure in the AiiDA database

In [ ]:
def on_click_store(b):
    if not orig_structure:
        print("No structure selected.")
        return

    nx = Int(nx_slider.value)
    ny = Int(ny_slider.value)
    nz = Int(nz_slider.value)
    dx = Float(dx_slider.value)
    dy = Float(dy_slider.value)
    dz = Float(dz_slider.value)
    phi = Float(phi_slider.value)
    s = prepare_mol_on_slab_wf(orig_structure,dx,dy,dz,phi, nx, ny, nz)
    s.description = inp_descr.value
    s.store()
    print("Stored in AiiDA: "+repr(s))

cell_ready = False
inp_descr = ipw.Text(placeholder="Description (optional)")   
btn_store = ipw.Button(description='Store in AiiDA')
btn_store.on_click(on_click_store)
display(ipw.HBox([btn_store, inp_descr]))

In [ ]:
@calcfunction
def modify_structure(orig_struct,list_modifications):
    modified_structure=None
    return StructureData(ase=new_atoms)